In [1]:
import os
train_label_path = '/home1/wsi/tct/train_smear.txt'
test_label_path = '/home1/wsi/tct/val_smear.txt'
output_path = 'labels'
file_names = ['train_label.csv', 'val_label.csv', 'test_label.csv', 'train_val.csv']


In [2]:
import csv
import shutil, os, random

random.seed(2024)

if not os.path.exists(output_path):
    os.mkdir(output_path)

output_paths = []
for i in range(len(file_names)):
    output_paths.append(os.path.join(output_path, file_names[i]))

## 训练集
with open(train_label_path, 'r') as f:
    train_lines = f.readlines()
    f.close()
train_ori_list = []
for line in train_lines:
    class_num = 0 if 'neg' in line else 1
    line = line.replace('neg/', '').replace('pos/', '').replace('\n', '')
    train_ori_list.append((line, class_num))
random.shuffle(train_ori_list)
train_list = train_ori_list[:int(len(train_ori_list)*0.8)]
val_list = train_ori_list[int(len(train_ori_list)*0.8):]

with open(output_paths[3], 'w') as f:
    writer = csv.writer(f)
    for line in train_ori_list:
        writer.writerow(line)
    f.close()
with open(output_paths[0], 'w') as f:
    writer = csv.writer(f)
    for line in train_list:
        writer.writerow(line)
    f.close()
with open(output_paths[1], 'w') as f:
    writer = csv.writer(f)
    for line in val_list:
        writer.writerow(line)
    f.close()


## 测试集
with open(test_label_path, 'r') as f:
    test_lines = f.readlines()
    f.close()
with open(output_paths[2], 'w') as f:
    writer = csv.writer(f)
    for line in test_lines:
        class_num = 0 if 'neg' in line else 1
        line = line.replace('neg/', '').replace('pos/', '').replace('\n', '')
        writer.writerow((line, class_num))
    f.close()
    
len_train, len_val, len_test = len(train_list), len(val_list), len(test_lines)
print(f'训练集: {len_train}, 验证集: {len_val}, 测试集: {len_test}')
print(f'总: {len_train + len_val + len_test}')

训练集: 1471, 验证集: 368, 测试集: 786
总: 2625


# sens.C

In [3]:
import os

label_dir = 'labels'
test_file = 'test_label.csv'
test_path = os.path.join(label_dir, test_file)
output_file = 'test_c.csv'
output_path = os.path.join(label_dir, output_file)

include_c_labels = ['LSIL', 'HSIL']

with open(test_path, 'r') as f:
    with open(output_path, 'w') as f_out:
        for line in f.readlines():
            wsi_name, wsi_label = line.strip().split(',')
            for label in include_c_labels:
                if label in wsi_name:
                    f_out.write(line)
    f_out.close()
f.close()


# Sens.H

In [4]:
import os

label_dir = 'labels'
test_file = 'test_label.csv'
test_path = os.path.join(label_dir, test_file)
output_file = 'test_h.csv'
output_path = os.path.join(label_dir, output_file)



include_h_labels = ['ASC-H', 'HSIL']

with open(test_path, 'r') as f:
    with open(output_path, 'w') as f_out:
        for line in f.readlines():
            wsi_name, wsi_label = line.strip().split(',')
            for label in include_h_labels:
                if label in wsi_name:
                    f_out.write(line)
    f_out.close()
f.close()


# 多分类

In [ ]:
import csv

subtypes = ['NILM', 'ASC-US', 'LSIL', 'ASC-H', 'HSIL', 'AGC']
two_label_dir = 'labels/'
n_label_dir = 'n-labels/'
sub_dirs=['train_val.csv','test_label.csv']

for sub_dir in sub_dirs:
    two_label_path = os.path.join(two_label_dir, sub_dir)
    n_label_path = os.path.join(n_label_dir, sub_dir)
    with open(two_label_path, 'r') as csvfile:
        print(f'==================================\033[35m{sub_dir}\033[0m==================================')
        with open(n_label_path, 'w') as n_csvfile:
            csv_reader = csv.reader(csvfile)
            csv_writer = csv.writer(n_csvfile)
            for row in csv_reader:
                wsi_name = row[0]
                if 'AGC' in wsi_name:
                    continue
                for i, subtype in enumerate(subtypes):
                    if subtype in wsi_name:
                        csv_writer.writerow([wsi_name, i])
                        break
                    if i == len(subtypes) - 1:
                        csv_writer.writerow([wsi_name, 0])
                        print('{} has not been classified, it has been classified as 0!'.format(wsi_name))
        n_csvfile.close()
    csvfile.close()